In [ ]:
%pip install -U langchain langchain-core langchain-community langchain-openai azure-search-documents azure-identity

In [ ]:
# 환경 설정 Import
from dotenv import load_dotenv

load_dotenv()

# KT 고객센터 상담 시나리오 FAQ - LCEL Chain 으로 만들기

In [ ]:
# 데이터 출처: https://ermsweb.kt.com/pc/faq/faqList.do
import json
category = [
    "USIM",
    "모바일",
    "혜택",
    "결합",
    "인터넷",
    "tv",
    "집전화",
    "인터넷전화",
    "IoT",
    "Egg",
    "와이파이",
    "기타",
    "SHOP",
]

def load_faq_data(file_path="faq_data.json"):
    """FAQ 데이터를 로드하는 함수"""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            result = json.load(f)
    except json.decoder.JSONDecodeError as e:
        print(f"[ERROR] JSONDecode: {e}")
    return result

# 데이터 로드
faq_data = load_faq_data()

## 1. FAQ 데이터 전처리

In [ ]:
# 1) Load and Parse 를 할 필요가 없음 -> 이미 다 파싱되어 JSON 으로 있습니다"
from pprint import pprint

pprint(category)
print("==" * 100)
pprint(faq_data)

## 1.1 LangChain Document 객체 이해

In [ ]:
from langchain_core.documents import Document

sample_doc = Document(
    page_content="이것은 문서의 내용입니다.\n실제 텍스트가 여기에 들어가게 되고, 이 내용 전체가 Embedding 대상인 Chunk 입니다.",
    metadata={
        "source": "faq",
        "category": "예제",
        "id": 1
    }
)

print("<<LangChain Document 구조>>")
print(f"- page_content: {sample_doc.page_content}")
print(f"- metadata: {sample_doc.metadata} \n  메타데이터는 Filtering 에 사용됩니다. (RDB 의 Where 조건절)")

## 2. FAQ 데이터 Embedding

In [ ]:
import os
from uuid import uuid4
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small",
    openai_api_version="2024-02-01",
)

#Azure AI Search(VectorDB)
vector_store = AzureSearch(
    azure_search_endpoint=f"https://{os.environ['AZURE_AI_SEARCH_SERVICE_NAME']}.search.windows.net",
    azure_search_key=os.environ["AZURE_AI_SEARCH_API_KEY"],
    index_name="faq-index",
    embedding_function=embeddings
)

print(embeddings)
print(vector_store)

In [ ]:
# JSON to Langchain Document
from langchain_core.documents import Document

def json_to_document(json_data: dict):
    documents = []
    for cat, qa_list in json_data.items():
        print(f"Category: {cat}")
        for _, qa_item in enumerate(qa_list):
                doc = Document(
                    page_content=f"Category: {cat}\nQuestion: {qa_item['question']}\nAnswer: {qa_item['answer']}",
                    metadata={
                        "category": cat,
                        "source": "faq"
                    }
                )
                documents.append(doc)
    return documents

docs = json_to_document(faq_data)
uuids = [str(uuid4()) for _ in range(len(docs))]

# Vector DB 에 저장
vector_store.add_documents(documents=docs, ids=uuids)

In [ ]:
# 단순 유사도 검색
from langchain_core.vectorstores import VectorStore

def search_simple_cos(input_vector_store: VectorStore, query: str, k: int = 3):
    _results = input_vector_store.similarity_search(query, k=k)
    return _results

test_question = "usim"
results = search_simple_cos(vector_store, test_question, k=1)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
# 단순 Cosine Similarity with Score (스코어까지 보여줌)

def search_simple_cos_with_score(input_vector_store: VectorStore, query: str, k: int = 3):
    _results = input_vector_store.similarity_search_with_score(query, k=k)
    return _results

score_question = "인터넷"
results = search_simple_cos_with_score(vector_store, score_question, k=1)
for res, score in results:
    print(f"* [SCORE={score:3f}]\n{res.page_content} [{res.metadata}]")

In [ ]:
# Retriever Customizing 실무용
def custom_retriever(k: int = 3, fetch_k: int = 5, filter: dict | None = None):
    result = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": fetch_k,
                       "filter": filter}
    )
    return result

_custom_r = custom_retriever()

real_question = "USIM가입 절차는?"
real_filter = {
    "category": "USIM가입"
}
_custom_r.invoke(real_question, filter=real_filter)

### 2.1 Hybrid Search (Semantic + **Lexical**)

In [ ]:
# Lexical Search Retriever
from langchain_community.retrievers import BM25Retriever

# 설치 필요
# %pip install rank_bm25

bm25_rr = BM25Retriever.from_documents(docs)
bm25_rr.invoke("인터넷 요금이 왜 이렇게 많이 나왔나요?", k=3)

In [ ]:
# 형태소 분석기 설치
# %pip install kiwipiepy konlpy

In [ ]:
# 그래서 어떤 Lexical(TF-IDF 기반) Search Retriever 를 쓰면 되는지?
from langchain.retrievers import EnsembleRetriever

ensemble_rr = EnsembleRetriever(
    retrievers=[
        bm25_rr,  # Lexical Retriever
        vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})  # Vector Retriever
    ],
    weights=[0.5, 0.5]  # 각 검색기의 가중치
)

In [ ]:
ensemble_rr.invoke("인터넷 요금이 왜 이렇게 많이 나왔나요?")

In [ ]:
# 추천하는 방식은 한글 형태소 분석기가 달린 BM25(TF-IDF 기반) 검색기를 사용을 기본으로 하는 Custom Retriever 를 만드는 것
# NOTE: 바로 가져다 쓰셔도 됩니다^^
from __future__ import annotations

from typing import Any, Callable, Dict, Iterable, List, Optional
from operator import itemgetter
import numpy as np

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from pydantic import Field
from langchain_core.retrievers import BaseRetriever

try:
    from kiwipiepy import Kiwi
except ImportError:
    raise ImportError(
        "Could not import kiwipiepy, please install with `pip install kiwipiepy`."
    )

kiwi_tokenizer = Kiwi()


def kiwi_preprocessing_func(text: str) -> List[str]:
    return [token.form for token in kiwi_tokenizer.tokenize(text)]


def default_preprocessing_func(text: str) -> List[str]:
    return text.split()


class KiwiBM25Retriever(BaseRetriever):
    """`BM25` retriever without Elasticsearch and Add Kiwi Tokenizer"""

    vectorizer: Any
    """ BM25 vectorizer."""
    docs: List[Document] = Field(repr=False)
    """ List of documents."""
    k: int = 5
    """ Number of documents to return."""
    preprocess_func: Callable[[str], List[str]] = kiwi_preprocessing_func
    """ Preprocessing function to use on the text before BM25 vectorization."""

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    @classmethod
    def from_texts(
        cls,
        texts: Iterable[str],
        meta_data: Optional[Iterable[dict]] = None,
        bm25_params: Optional[Dict[str, Any]] = None,
        preprocess_func: Callable[[str], List[str]] = kiwi_preprocessing_func,
        **kwargs: Any,
    ) -> KiwiBM25Retriever:
        """
        Create a KiwiBM25Retriever from a list of texts.
        Args:
            texts: A list of texts to vectorize.
            meta_data: A list of metadata dicts to associate with each text.
            bm25_params: Parameters to pass to the BM25 vectorizer.
            preprocess_func: A function to preprocess each text before vectorization.
            **kwargs: Any other arguments to pass to the retriever.

        Returns:
            A KiwiBM25Retriever instance.
        """
        try:
            from rank_bm25 import BM25Okapi
        except ImportError:
            raise ImportError(
                "Could not import rank_bm25, please install with `pip install "
                "rank_bm25`."
            )

        texts_processed = [preprocess_func(t) for t in texts]
        bm25_params = bm25_params or {}
        vectorizer = BM25Okapi(texts_processed, **bm25_params)
        meta_data = meta_data or ({} for _ in texts)
        docs = [Document(page_content=t, metadata=m) for t, m in zip(texts, meta_data)]
        return cls(
            vectorizer=vectorizer, docs=docs, preprocess_func=preprocess_func, **kwargs
        )

    @classmethod
    def from_documents(
        cls,
        documents: Iterable[Document],
        *,
        bm25_params: Optional[Dict[str, Any]] = None,
        preprocess_func: Callable[[str], List[str]] = kiwi_preprocessing_func,
        **kwargs: Any,
    ) -> KiwiBM25Retriever:
        """
        Create a KiwiBM25Retriever from a list of Documents.
        Args:
            documents: A list of Documents to vectorize.
            bm25_params: Parameters to pass to the BM25 vectorizer.
            preprocess_func: A function to preprocess each text before vectorization.
            **kwargs: Any other arguments to pass to the retriever.

        Returns:
            A KiwiBM25Retriever instance.
        """
        texts, meta_data = zip(*((d.page_content, d.metadata) for d in documents))
        return cls.from_texts(
            texts=texts,
            bm25_params=bm25_params,
            metadatas=meta_data,
            preprocess_func=preprocess_func,
            **kwargs,
        )

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        processed_query = self.preprocess_func(query)
        return_docs = self.vectorizer.get_top_n(processed_query, self.docs, n=self.k)
        return return_docs

    @staticmethod
    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    @staticmethod
    def argsort(seq, reverse):
        return sorted(range(len(seq)), key=seq.__getitem__, reverse=reverse)

    def search_with_score(self, query: str, top_k=None):
        normalized_score = KiwiBM25Retriever.softmax(
            self.vectorizer.get_scores(self.preprocess_func(query))
        )

        if top_k is None:
            top_k = self.k

        score_indexes = KiwiBM25Retriever.argsort(normalized_score, True)

        docs_with_scores = []
        for i, doc in enumerate(self.docs):
            document = Document(
                page_content=doc.page_content, metadata={"score": normalized_score[i]}
            )
            docs_with_scores.append(document)

        score_indexes = score_indexes[:top_k]

        # Creating an itemgetter object
        getter = itemgetter(*score_indexes)

        # Using itemgetter to get items
        selected_elements = getter(docs_with_scores)
        return selected_elements

In [ ]:
# 그럼 최종적으로 Hybrid Search Retriever 를 EnsembleRetriever 로 만들어본다면?
hybrid_retriever = EnsembleRetriever(
    retrievers=[
        KiwiBM25Retriever.from_documents(docs),  # Kiwi BM25 Retriever
        vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})  # Vector Retriever
    ],
    weights=[0.7, 0.3]  # 각 검색기의 가중치
)


hybrid_retriever.invoke("인터넷 요금이 왜 이렇게 많이 나왔나요?")

## 2.2 텍스트 분할(Text Splitting)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,      # 청크 크기(500 이하로 추천)
    chunk_overlap=50,    # 청크 간 겹침
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # 분할 우선 순위
)

# 문서 분할
split_docs = text_splitter.split_documents(docs)
print(f"✅ {len(docs)}개 문서 → {len(split_docs)}개 청크로 분할")

# 분할 결과 확인
print("\n🔍 분할 예시:")
print(f"원본 문서 길이: {len(docs[0].page_content)}자")
if len(split_docs) > len(docs):
    print(f"분할된 첫 번째 청크: {split_docs[0].page_content}")

### 3. FAQ LCEL Chain 만들기

In [ ]:
def format_docs(docs: list[Document]) -> str:
    """
    검색된 문서들을 하나의 문자열로 포맷팅

    Args:
        docs: Document 객체 리스트
    """
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"[Document Context_{i}]\n{doc.page_content}]")

    return "\n\n".join(formatted)

In [ ]:
sys_prompt = """당신은 KT 고객센터 상담원입니다.
무조건 아래 참고 자료를 기반으로 하여 고객의 질문에 친절하고 정확하게 제공된 자료만을 사용하여 답변하세요.

-----
참고 자료:
{context}
-----
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

messages = [
    SystemMessagePromptTemplate.from_template(sys_prompt),
    HumanMessagePromptTemplate.from_template("{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, ChatOpenAI

###
deployment = "gpt-4.1-nano" # 모델 Deployment 이름
api_version="2025-01-01-preview"
###

rag_model = AzureChatOpenAI(
    azure_deployment=deployment,
    api_version=api_version,
    temperature=0.1,
)

In [ ]:
# Chain 생성
from langchain_core.runnables import RunnablePassthrough

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

naive_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | rag_model

naive_chain

## 4. FAQ Chain 실행

In [ ]:
user_question = "인터넷이 자꾸 끊겨요. 어떻게 해야 하나요?"
response = naive_chain.invoke(user_question)

response.pretty_print()

## 5. FAQ Chain 업그레이드

### 5.1 Few-shot Prompt

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "휴대폰 요금이 너무 비싸요",
        "output": """고객님, 휴대폰 요금 부담을 덜어드릴 수 있는 방법을 안내해드리겠습니다.

1. **요금제 변경**: 현재 사용량에 맞는 요금제로 변경하시면 절약 가능합니다.
2. **가족 결합**: 가족과 함께 결합하시면 추가 할인을 받으실 수 있습니다.
3. **멤버십 할인**: KT 멤버십 등급에 따라 할인 혜택이 있습니다.

고객님의 현재 요금제와 사용 패턴을 확인하여 최적의 방안을 찾아드리겠습니다.
추가로 궁금하신 점이 있으시면 말씀해 주세요."""
    },
    {
        "input": "인터넷이 안 돼요",
        "output": """고객님, 인터넷 연결 문제로 불편을 드려 죄송합니다.
다음 단계를 순서대로 확인해 주세요:

1. **모뎀/공유기 확인**
   - 전원이 켜져 있는지 확인
   - 모든 케이블이 제대로 연결되어 있는지 확인

2. **재시작**
   - 모뎀과 공유기의 전원을 뺐다가 30초 후 다시 연결

3. **기기 설정**
   - WiFi가 켜져 있는지 확인
   - 올바른 네트워크에 연결되어 있는지 확인

위 방법으로도 해결되지 않으면 고객센터(100번)로 연락 주시면
원격 점검을 도와드리겠습니다."""
    }
]

# Few-shot 프롬프트 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 10년 경력의 친절한 KT 고객센터 상담원입니다."),
    few_shot_prompt,
    ("human", """참고 자료:
{context}

고객 질문: {question}""")
])

# Few-shot RAG 체인
few_shot_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | final_prompt
    | rag_model
)
few_shot_rag_chain

In [ ]:
few_shot_test_question = "내 인터넷 요금이 왜 이렇게 많이 나왔나요?"
few_shot_response = few_shot_rag_chain.invoke(few_shot_test_question)

few_shot_response.pretty_print()

### 5.2 메모리

In [ ]:
# 주의: 현재 LangChain 에서 제공하는 Conversation Memory 는 모두 Deprecated 되었습니다.
# Conversation 메모리의 개념을 이해하고 활용하는 것으로만 활용하시고, 실제 업무에는 사용하시면 안됩니다.



## 6. Chain 전략 다변화

### 6.1 질문 의도 분석 체인 추가

In [ ]:
# 의도 분석 프롬프트
intent_analysis_sys_prompt = """
당신은 KT 고객센터의 상담 전문가이면서 의도 분석에 관한 최고 전문가입니다.
고객의 질문을 깊이 있게 한국어의 맥락을 차분히 파악, 분석하여 정확한 의도를 파악해주세요.

## 분석 기준:
1. 주요 의도: 고객이 원하는 핵심 목적
2. 긴급도: 문제의 시급성 (high/medium/low)
3. 감정: 고객의 감정 상태 (positive/neutral/negative)
4. 상담원 필요: 자동 응답으로 해결이 불가능한 경우


## 의도 카테고리는 다음과 같습니다:
{intent_categories}

## Output Format:
{format_instructions}
"""

intent_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", intent_analysis_sys_prompt),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

from enum import Enum

class UserIntent(str, Enum):
    """사용자 의도 분류"""
    INFO_INQUIRY = "info_inquiry"              # 정보 문의
    SERVICE_APPLICATION = "service_application" # 서비스 신청
    SERVICE_CHANGE = "service_change"          # 서비스 변경
    SERVICE_CANCELLATION = "service_cancellation" # 서비스 해지
    BILLING_INQUIRY = "billing_inquiry"        # 요금 문의
    TECHNICAL_SUPPORT = "technical_support"    # 기술 지원
    COMPLAINT = "complaint"                    # 불만 사항
    GENERAL_INQUIRY = "general_inquiry"        # 일반 문의

class IntentAnalysis(BaseModel):
    """의도 분석 결과"""
    intent: UserIntent = Field(description="주요 의도")
    sub_intent: str | None = Field(None, description="세부 의도")
    confidence: float = Field(description="신뢰도 (0.0-1.0)", ge=0.0, le=1.0)
    keywords: list[str] = Field(description="핵심 키워드(5개 이하)")
    urgency: Literal["high", "medium", "low"] = Field(description="긴급도")
    sentiment: Literal["positive", "neutral", "negative"] = Field(description="감정 상태")
    requires_human: bool = Field(description="상담원 연결 필요 여부")


intent_parser = JsonOutputParser(pydantic_object=IntentAnalysis)

In [ ]:
# 의도 분석 체인
intent_analysis_chain = (
    {
        "question": RunnablePassthrough(),
        "format_instructions": lambda x: intent_parser.get_format_instructions(),
        "intent_categories": lambda x: ", ".join([i.value for i in UserIntent])
    }
    | intent_chat_prompt
    | rag_model
    | intent_parser
)
intent_analysis_chain

### 6.2 감정 분석 체인 추가

In [ ]:
sentiment_analysis_sys_prompt = """당신은 고객의 감정을 분석하는 전문가입니다."""
sentiment_analysis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sentiment_analysis_sys_prompt),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

### 6.3 질문 의도 + 감정 분석 병렬 처리

In [ ]:
# TODO: 실습!

### 6.4 각 의도별로 전문화된 체인 만들기

In [ ]:
# 각 의도별 전문 체인 (예시)

billing_chain = ChatPromptTemplate.from_template(
    "요금 전문 상담원입니다. 요금 관련 문의: {question}"
) | rag_model

technical_chain = ChatPromptTemplate.from_template(
    "기술 지원팀입니다. 기술적 문제: {question}\n단계별 해결 방법을 안내하겠습니다."
) | rag_model

service_chain = ChatPromptTemplate.from_template(
    "서비스 가입/변경 담당입니다. 문의 사항: {question}"
) | rag_model

cancellation_chain = ChatPromptTemplate.from_template(
    "해지 방지 팀입니다. 고객님의 불편사항: {question}\n혜택을 안내드리겠습니다."
) | rag_model

general_chain = ChatPromptTemplate.from_template(
    "일반 상담원입니다. 문의사항: {question}"
) | rag_model

In [ ]:
# 의도 기반 라우팅 체인
from langchain_core.runnables import RunnableBranch

def classify_intent(question: str) -> str:
    # TODO: 과연 어떻게 만들어야 의도를 키워드로 추출할 수 있을까요?
    return "general"

intent_router = RunnableBranch(
    (lambda x: classify_intent(x["question"]) == "billing", billing_chain),
    (lambda x: classify_intent(x["question"]) == "technical", technical_chain),
    (lambda x: classify_intent(x["question"]) == "service", service_chain),
    (lambda x: classify_intent(x["question"]) == "cancellation", cancellation_chain),
    general_chain  # 기본값
)


### 6.5 고객 정보 추가

In [ ]:
from pydantic import BaseModel, Field

class CustomerContext(BaseModel):
    """고객 컨텍스트"""
    customer_type: str = Field(description="고객 유형: VIP/일반/신규")
    subscription_months: int = Field(description="가입 기간(월)")
    monthly_fee: int = Field(description="월 요금")
    has_issues: bool = Field(description="최근 문제 발생 여부")

In [ ]:
def create_customer_aware_chain():
    from langchain_core.output_parsers import StrOutputParser
    """고객 인식 체인 생성"""

    # VIP 전용 체인
    vip_chain = ChatPromptTemplate.from_template(
        """VIP 고객님께 특별한 서비스를 제공합니다.
        가입 기간: {subscription_months}개월
        월 요금: {monthly_fee:,}원

        문의사항: {question}

        VIP 전용 혜택과 함께 최우선으로 처리해드리겠습니다."""
    ) | rag_model | StrOutputParser()

    # 이탈 위험 고객 체인
    churn_risk_chain = ChatPromptTemplate.from_template(
        """소중한 고객님, 불편을 드려 죄송합니다.
        고객님께서 겪으신 문제를 해결하고 특별 혜택을 제공하겠습니다.

        문의사항: {question}

        고객님을 위한 맞춤 혜택을 준비했습니다."""
    ) | rag_model | StrOutputParser()

    # 일반 고객 체인
    normal_chain = ChatPromptTemplate.from_template(
        """안녕하세요, KT입니다.

        문의사항: {question}

        도움을 드리겠습니다."""
    ) | rag_model | StrOutputParser()

    # 조건부 라우팅
    return RunnableBranch(
        # VIP 고객
        (lambda x: x.get("customer_type") == "VIP", vip_chain),
        # 이탈 위험 고객 (오래된 고객 + 최근 문제)
        (lambda x: x.get("subscription_months", 0) > 24 and x.get("has_issues", False), churn_risk_chain),
        # 기본
        normal_chain
    )

# 고객 인식 체인 생성
customer_aware_chain = create_customer_aware_chain()

# 다양한 고객 시나리오 테스트
test_customers = [
    {
        "customer_type": "VIP",
        "subscription_months": 60,
        "monthly_fee": 150000,
        "has_issues": False,
        "question": "해외 로밍 요금이 궁금합니다"
    },
    {
        "customer_type": "일반",
        "subscription_months": 36,
        "monthly_fee": 50000,
        "has_issues": True,
        "question": "서비스가 불만족스러워서 해지를 고려중입니다"
    },
    {
        "customer_type": "신규",
        "subscription_months": 2,
        "monthly_fee": 35000,
        "has_issues": False,
        "question": "5G 요금제로 변경하고 싶어요"
    }
]

print("👥 고객별 맞춤 응답:\n")
for customer in test_customers:
    print(f"고객 유형: {customer['customer_type']} (가입 {customer['subscription_months']}개월)")
    print(f"Q: {customer['question']}")
    response = customer_aware_chain.invoke(customer)
    print(f"A: {response[:150]}...")
    print("=" * 70)



# 의도별 응답 템플릿
class IntentRouter:
    """의도별 응답 라우터"""

    def __init__(self, llm):
        self.llm = llm
        self.templates = self._create_templates()

    def _create_templates(self) -> Dict[UserIntent, ChatPromptTemplate]:
        """의도별 전문 템플릿 생성"""
        return {
            UserIntent.BILLING_INQUIRY: ChatPromptTemplate.from_template(
                """요금 전문 상담원입니다.
                질문: {question}
                분석: {analysis}

                정확한 요금 정보와 절약 방법을 안내해드리겠습니다."""
            ),

            UserIntent.TECHNICAL_SUPPORT: ChatPromptTemplate.from_template(
                """기술 지원팀입니다.
                질문: {question}
                긴급도: {urgency}

                단계별 해결 방법을 안내하겠습니다."""
            ),

            UserIntent.COMPLAINT: ChatPromptTemplate.from_template(
                """고객님의 불편을 해결하겠습니다.
                질문: {question}
                감정: {sentiment}

                진심으로 사과드리며 즉시 개선하겠습니다."""
            ),

            UserIntent.SERVICE_CHANGE: ChatPromptTemplate.from_template(
                """서비스 변경을 도와드리겠습니다.
                질문: {question}

                최적의 서비스로 안내해드리겠습니다."""
            )
        }

    def route_and_respond(self, question: str, analysis: IntentAnalysis) -> str:
        """의도에 따라 적절한 응답 생성"""
        # 기본 템플릿
        default_template = ChatPromptTemplate.from_template(
            "KT입니다. {question}에 대해 도움을 드리겠습니다."
        )

        # 의도별 템플릿 선택
        template = self.templates.get(analysis.intent, default_template)

        # 체인 실행
        chain = template | self.llm

        response = chain.invoke({
            "question": question,
            "analysis": analysis.model_dump_json(),
            "urgency": analysis.urgency,
            "sentiment": analysis.sentiment
        })

        return response.content

## 정리

1) 입력에 대해 전처리 및 분석
2) 병렬 분석 및 데이터 처리
3) Routing 결정
4) 최종 RAG Chain 확정

### 문제점
순차적인 진행(단방향), 1개라도 실패하면 되돌아올 수 없음(전체 실패)